In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta

from pynori.korean_analyzer import KoreanAnalyzer

import time
import schedule


es = Elasticsearch('http://127.0.0.1:9200')

#키워드는 "빅데이터","빅데이타" 연관검색어는 1개만 넣는것이 좋음 (중복된 데이터가 수집됨) 
competitor=['카카오','삼성sds','lgcns','디리아','메가존클라우드','nshc',"cloudera","엘라스틱서치코리아","네이버","이지팜"]
bigdata_industry = ["빅데이터", "마이데이터", "가트너","DW", "데이터웨어하우스", "Migration", "ETL","EAI","ESB","cloud","AWS","GCP"]
bigdata_solution = ["splunk","apache kafka", "apache spark", "elasticsearch","zookeeper", "confluent","yarn","kibana","logstash","file beats","docker","kubernetes"]
keyword = [competitor , bigdata_industry , bigdata_solution]

now = datetime.now()
today = now.strftime('%Y-%m-%d')
yesterday = now - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d-%H-%M')

title_list = []
url_list = []
comp_list = []
thumbnail_list = []
text_list = []
day_list = []
keyword_list =[]
category_list = []
keyword = sum(keyword,[])

# python과 elasticsearch의 nori를 합쳐서 만든 새로운 패키지 pynori(2020.07 생겨남), pynori에서 KoreanAnalyzer 패키지 사용
# 밑의 코드는 문장에서 '명사'만 골라오는 코드이다.
nori = KoreanAnalyzer(
           decompound_mode='DISCARD', # DISCARD or MIXED or NONE
           infl_decompound_mode='DISCARD', # DISCARD or MIXED or NONE
           discard_punctuation=True,
           output_unknown_unigrams=False,
           pos_filter=True, stop_tags=['JKS',
                                       'JKB',
                                       'VV',
                                       'EF',
                                       'EP',
                                       'JX',
                                       'VA',
                                       'ETM',
                                       'VCP',
                                       "E",
                                      "IC",
                                      "J",
                                      "MAG",
                                      "MAJ",
                                      "MM",
                                      "NR",
                                      "NA",
                                      "SC",
                                      "SE",
                                      "SF",
                                      "SH",
                                      "SN",
                                      "SP",
                                      "SSC",
                                      "SSO",
                                      "SY",
                                      "UNA",
                                      "UNKNOWN",
                                      "VA",
                                      "VCN",
                                      "VCP",
                                      "VSV",
                                      "VV",
                                      "VX",
                                      "XPN",
                                      "XR",
                                      "XSA",
                                      "XSN",
                                      "XSV",
                                      "NNB",
                                      "NNBC",
                                      "EC",
                                      "JKO",
                                      "NP"
                                      ],
           synonym_filter=False, mode_synonym='NORM', # NORM or EXTENSION
       ) 

def printhello():
    
    for j in range(len(competitor)+len(bigdata_industry)+len(bigdata_solution)) :


        response1 = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query='+ keyword[j]
        response2= '&sort=0&photo=0&field=0&pd=4&ds=' + today + '&de=' + yesterday + '&cluster_rank=27&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:1d,a:all&start='

        response = requests.get(response1+response2+str(1))
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        articles = soup.select('#main_pack > section > div > div.group_news > ul > li')


        for article in articles:
            global title_list
            
            a_tag1 = article.select_one('.news_tit')

            title = a_tag1.text
            title = nori.do_analysis(title)['termAtt']  # pynori로 분석해주기 
            title_list.append(title)                    # 이중 list꼴로 되어 있음

            url = a_tag1['href']
            url_list.append(url.strip())

            comp = article.select_one('a.info.press').text
            comp = comp.replace('언론사 선정', '')
            comp_list.append(comp.strip())

            text = article.select_one('a.api_txt_lines.dsc_txt_wrap').text
            text_list.append(text.strip())

            day_list.append(today)

            keyword_list.append(keyword[j])

            try:
                thumbnail = article.select_one('div > a > img')['src']
                thumbnail_list.append(thumbnail.strip())

            except:
                thumbnail_list.append(
                    'https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F366%2F2021%2F09%2F23%2F762207.jpg&type=ff264_180&expire=2&refresh=true')
            # 일부 기사는 이미지가 없는 경우가 있다. 따라서 이미지가 없는 경우 기본값을 설정해주었다.


            if (keyword[j] in competitor):
                category_list.append("competitor")

            elif (keyword[j] in bigdata_industry):
                category_list.append("bigdata_industry")

            elif (keyword[j] in bigdata_solution):
                category_list.append("bigdata_solution")               
    
    titles_list = []
    for element in title_list:
        titles_list += element    # title_list 이중 list꼴 풀어주기
                             
    info = {'title':titles_list, 'url':url_list, 'company':comp_list, 'contents':text_list, 'date':day_list, 'image':thumbnail_list ,'keyword':keyword_list, 'category': category_list}

    print(info)
    
    
#     index_name = 'kmh_test2'+"_"+today

#     for i in range(len(info['title'])):
#         es.index(index=index_name, 
#             doc_type='_doc', 
#             body={'title':info['title'][i],
#             'url':info['url'][i], 
#             'company':info['company'][i],
#             'contents':info['contents'][i], 
#             'date':info['date'][i],
#             'image':info['image'][i],
#             'keyword':info['keyword'][i],
#             'category':info['category'][i]})

#     es.indices.put_alias(index = index_name, name = 'kmh_test2')
    
schedule.every(10).seconds.do(printhello) #10초마다 실행
schedule.every().monday.at("00:10").do(printhello) #월요일 00:10분에 실행
schedule.every().day.at("10:30").do(printhello) #매일 10시 30분에  

while True:
    schedule.run_pending()
    time.sleep(1)

{'title': ['카카오', '먹', '거리', '발굴', '김범수', '남궁훈', '톱', '카카오', '게임즈', '남궁훈', '카카오', '시즌', '카카오', '네이버', '웹', '협', '웹툰', '불법', '유통', '뿌리', '카카오', '워크', '누적', '가입', '자', '돌파', '협업', '툴', '시장', '안착', '상장', '시동', '카카오', '모빌리티', '우티', '택시', '여론', '관건', '골프', '보험', '카카오', '사업', '확장', '문어', '발', '메기', '속보', '네이버', '카카오', '맵', '카카오', '내비', '소수', '재고', '가격', '제공', '카카오', '뱅크', '내년', 'atm', '수수', '료', '면제', '구', '글', '韓', '앱', '시장', '영향', '력', '카카오', '네이버', 'gs', '마이', '샵', '카카오', 'vx', '골프', '연습', '장', '창업', '상담', '방송', '삼성', 'sds', '주가', '삼성', 'sdi', '삼성', '전기', 'lg', '권봉', '석', 'sk', '장동현', '삼성', '고동', '진', 'ceo', '출신', '중', '징계', '결론', '해외', '자', '회사', '인수', '승인', '삼성', '생명', '징계', '수위', 'it', '서비스', '분기', '실적', '줌', '이종목', '데이터', '보안', '강자', '파수', '이', '달', '상승', '테크', 'm', '김범수', '의', '복심', '남궁훈', '집', '살림', '스타트', '카카오', '시즌', '도', '벨', '신한', '카드', '데이터', '전문기', '관업', '진출', '노크', 'lg', 'cns', '클라우드', '스마트', '물류', '금융', 'it', '사업', '부로', '격상', 'lg', '전자', '해외', '법인', '콜', '센터', '클라우드', '전환',

KeyboardInterrupt: 